Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [11]:
import numpy as np
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [12]:
# построчное чтение из примера с текстом 
with open("alisa_russian.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}



# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE = 128
HIDDEN_SIZE_L1 = 512
HIDDEN_SIZE_L2 = 256
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100
DROPOUT_RATE = 0.2
TEMPERATURE = 0.5

In [14]:
'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE_L1,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
# model.add(Dropout(DROPOUT_RATE))

model.add(LSTM(HIDDEN_SIZE_L2))
# model.add(Dropout(DROPOUT_RATE))

model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")

In [15]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        # next_index = sample(pred, TEMPERATURE)
        # y_pred = index2char[next_index]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/3
167947/167947 [==============================] - 33s 198us/step - loss: 2.5460
Epoch 2/3
167947/167947 [==============================] - 32s 191us/step - loss: 1.9755
Epoch 3/3
167947/167947 [==============================] - 31s 187us/step - loss: 1.7454
Генерация из посева: .- И тут Д
.- И тут Деликатес в конце не принялась она, не все все все все все все все все все все все все все все все вс==================================================
Итерация #: 1
Epoch 1/3
167947/167947 [==============================] - 32s 188us/step - loss: 1.5873
Epoch 2/3
167947/167947 [==============================] - 32s 188us/step - loss: 1.4583
Epoch 3/3
167947/167947 [==============================] - 31s 183us/step - loss: 1.3338
Генерация из посева: столько ре
столько ребеники и получилась вспомнить скольедное присяжных изопртной кости стороным подобротельно присяжных ==================================================
Итерация #: 2
Epoch 1/3
167947/167947 [=============

**Несколько результатов, которые мне понравились:**

- роль сердито.- Остроумная шутка! Тут все захохотали. . - Удаляйтесь на совещание! - сказала Алиса. - А самое у
- ожно перечитать непонятное место еще разок, правда? Только вот кто подпоет? - Пой уж ты,- сказал Король. Тут в
- не осталось, а все, кто сидел у входа. - Давайте подвинемся! Он тут же пересел на соседний стул; Соня села пр
- ательно случится что-нибудь попроще,- великодушно предложил Червяк. Вопрос был для Алисы совершенно правильные

Для того, чтобы получить такой результат я увеличил количество слоев LSTM до 2, этох до 3 и количество нейронов в них до 256. Пробовал добавить дропауты и температуру, но это не  улучшило результаты, поэтому от этого отказался.